First of all, let's import the libraries we'll be working with

In [ ]:
import pandas as pd
from pandas import DataFrame

Now we will create a functions that we will use throughout the code

- First of it will be function that will show the head of the table

In [ ]:
def show_head(df):
    return df.head()

- this function will show the datatypes of variables

In [ ]:
def show_data_type(df):
    return df.dtypes

- and the third will show the general info about datasets

In [ ]:
def show_info(df):
    return df.info()

0. To start with, I have read the file to a dataframe.

In [ ]:
show_head(pd.read_excel('/home/master/pycharm_projects/amazon_products/data/original/Amazon.xlsx'))

I wanted to skip the title and the empty row in order to have a better view.

In [ ]:
df = pd.read_excel('/home/master/pycharm_projects/amazon_products/data/original/Amazon.xlsx', header=None, skiprows=3)

let's check the changed data

In [ ]:
show_head(df)

after the checking, it seems good. To check the datatype we first name columns

In [ ]:
df.columns = ["Category", "Product ID", "Year", "Quarter", "Quarterly Sales", "Customer Rating", "Seller Rating", "Price", "Discount Rate", "Units Sold", "Revenue", "Shipping Method", "Location", "Advertising Cost"]

let's check the changed names of variables 

In [ ]:
show_head(df)

1. examine the variables and their datatypes

In [ ]:
show_data_type(df)

there seems to be problem with Discount Rate and Seller Rating, they should ve integer instead of floats. We are going to do this change later since it will be much easier

2. remove any separation error in variables.

since we defined the header there is no need to check the existence of it check the exising of NA's

In [ ]:
df.isna().sum()

it is showed that there are NA's values in Seller Rating, Advertising Cost, Quarterly Sales

for making sure that there isn't any NA type of string type let's first of all get rid of white spaces in the code

In [ ]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

and replacing string NA by NA

In [ ]:
df.replace('NA', pd.NA, inplace=True)

let's check whether if there are new NA 

In [ ]:
df.isna().sum()

so, there was 1 string NA in column "Quarterly Sales"  

so, there is a problem only in The column: Quarterly Sales, we will consider it later

Remove specified characters from string columns

In [ ]:
df.replace(to_replace=[r'\{', r'\}', r';', r':', r',', r'\t', r'\n'], value='', regex=True, inplace=True)


3. since we define by ourselves the names of columns there is no need to check whether column headers are values, not variables names

- for checking whether multiple variables are stored in both rows and columns. First of all, let's find out whether multiple datatypes present in a column

In [ ]:
df.applymap(type).apply(set)

so, we can see that in Quarterly Sales there are three types of value, we will fix it later, since we need to first of all find out what shall we do with NA's

Check if specific values indicate multiple variables in all columns

creating the function that will check for presenting of a specific characters and after this remove it 

In [ ]:
def check_and_remove_specific_values(df, specific_values):
    return df.applymap(lambda x: ''.join(c for c in str(x) if c not in specific_values))

let's check the columns

In [ ]:
# Define a list of specific values to check for and remove
specific_values = [':', ';', ',', '.', '/', '?', '|', '*', '%']

# Check for the presence of specific values in all columns
check_and_remove_specific_values(df, specific_values)

so, right now we have the data that don't have any if these specific charters 

 let's check whether each string in each column one or more word characters let's check the pattern letter and one digit ot more digits Product ID

In [ ]:
pattern_to_check_product_id = '^[A-Z]\d+$'
# Check if the pattern is present in Product ID column

df['Product ID'] = df['Product ID'].astype(str)  # Ensure the column is of string type
df['Product ID'].str.match(pattern_to_check_product_id)

it seems that everything is ok 

check whether quarter column has a patter Q and after this digit from 1 to 4 

In [ ]:
pattern_to_check_quarter = '^Q[1-4]$'

# Check if the pattern is present in Quarter column
df['Quarter'] = df['Quarter'].astype(str)  # Ensure the column is of string type
df['Quarter'].str.match(pattern_to_check_quarter)

it seems that everything is ok 

let's check the others categorical columns that they contain only letters 

In [ ]:
# Identify categorical columns (excluding 'Product ID' and 'Quarter')
categorical_columns = df.select_dtypes(include='object').columns.difference(['Product ID', 'Quarter', 'Quarterly Sales'])

# Check if categorical columns contain only digits
for column in categorical_columns:
    df[column] = df[column].astype(str)  # Ensure the column is of string type
    contains_only_digits = df[column].str.match(r'\w+')
    print(f"{column} contains only digits:")
    print(contains_only_digits)

Column Category, Location, Shipping method contains only letters they are Ok

other columns should be check later since they contain NA values that should be evaluated 

Check if variables are stored in both rows and columns

In [ ]:
set(df.columns) & set(df.values.flatten())

so it is shown the empty set it means none of variables are stored in both rows and columns

since we have one table there is no need to check for whether a single observational unit exist in a different tables, only drop duplicates

In [ ]:
df.drop_duplicates(inplace=True)

4. we already fixed the column names

5. The product ID column is unnecessary for data visualization, so I'm deleting it.

In [ ]:
df = df.drop("Product ID", axis=1)

6. we already clean data from duplicates 

7. we already check if we have any unnecessary strings in the values.

8. we already check for whitespaces

9. we need to format a titles

In [ ]:
df["Category"] = df["Category"].str.title()

10. We don't have to check if levels of categories are unique since we don't need it to be unique for this data set.

11. we need to create a column that would specify year and quarter but firstly we need to work with Year column.

some data has at the end uneceasty one more digit let's get rid of id

In [ ]:
df['Year'] = df['Year'].astype(str)
df['Year'] = df['Year'].str[:4]
df['Year'] = df['Year'].astype(int)

right now we can create a new column

In [ ]:
# Function to create date range for each row
def generate_date(row):
    quarter_map = {'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}
    start_date_str = f"{row['Year']}-{quarter_map[row['Quarter']]}"
    start_date = pd.to_datetime(start_date_str, format="%Y-%m")
    end_date = start_date + pd.DateOffset(months=2)
    return pd.date_range(start=start_date, end=end_date, freq='M')
# Apply the function to create the 'Date' column
df['Date'] = df.apply(generate_date, axis=1)

# Explode the 'Date' column to handle the date range
df = df.explode('Date').reset_index(drop=True)

# Display the DataFrame
df["Date"]

we created a new column that will have date format with specifying purchases in the quarter

so, as we checked we NA's in Quarterly Sales and in Seller Rating

we've found that values in Quarterly Sales is Revenue divide on Units Sold. So, instead of replacing all data by the mean/median/mode we can just replace it by Revenue divide on Units Sold

In [ ]:
df['Quarterly Sales'] = df['Quarterly Sales'].fillna(df["Revenue"] // df["Units Sold"])
df['Quarterly Sales'] = df['Quarterly Sales'].astype(int)

let's see if there are a NA in column

In [ ]:
df["Quarterly Sales"].isna().sum()

so, there isn't any NA's

let's the data in this column

In [ ]:
sorted_df = df.sort_values(by='Category')
sorted_df[["Category", "Quarterly Sales"]]

median of rating since all data in integer we should use meidan

In [ ]:
df['Seller Rating'].fillna(df['Seller Rating'].median(), inplace=True)

the same with advertising cost 

In [ ]:
df['Advertising Cost'].fillna(df['Advertising Cost'].mean(), inplace=True)

some data in year is typed in a wring for example instead of 2019 there is written 20199

In [ ]:
df['Year'] = df["Year"].astype(str)
df['Year'] = df['Year'].str[:4]
df['Year'] = df["Year"].astype(int)

let's review our change 

In [ ]:
df['Year']

let's check whether in numeric exist negative, in our data it is unlikely if there would exist such values    

In [ ]:
negative_values_columns = df.select_dtypes(include='number').columns[df.select_dtypes(include='number').lt(0).any()]

# Display the result
if not negative_values_columns.empty:
    print(f"Negative values exist in the following columns: {', '.join(negative_values_columns)}")
else:
    print("No negative values found in any numeric column.")

range of rating is unlikely to be negative since almost all data is ranging
from 0 to 5. we should change the negative values to positive one also convert all data to integer since all data is integer numbers

In [ ]:
df["Customer Rating"] = df["Customer Rating"].abs().astype(int)

let's review the column

In [ ]:
(df['Customer Rating'] < 0).any()

it shows that everything is OK

make all data inn columns in one type

In [ ]:
df['Category'] = df['Category'].replace({'BEAUTY': 'Beauty',
                                         'ELECTRONICS': 'Electronics',
                                         'Home - Kitchen': 'Home & Kitchen',
                                         'Book': 'Books'})
df['Location'] = df['Location'].replace({'United States': 'US', 'ASIA': 'Asia', 'European Union': 'EU'})

let's check for outliers. It would be more logical if we will detect outliers in each column based on category since if we will change outliers based only on colum we might replace the data that corresponds to one category. Furthermore, average price might be 10 $ bur for the car might be 100000 $ and it would be unwise if we replace some of the by the median of column.   

Let's write the function that will detect outliers based on Box-plot. Since each category isn't large we should detect extreme outliers  

In [ ]:
def identify_outliers_by_category(df, category_column, numerical_column):
    categories = df[category_column].unique()
    outliers_dict = {}

    for category in categories:
        subset = df[df[category_column] == category]
        column = subset[numerical_column]

        Q1 = column.quantile(0.25)
        Q3 = column.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        outliers = column[(column < lower_bound) | (column > upper_bound)]

        # Check if the number of outliers doesn't exceed 10% of the category's data
        if not outliers.empty:
            category_count = len(subset)
            outlier_count = len(outliers)
            if outlier_count / category_count <= 0.1:
                outliers_dict[category] = outliers

    return outliers_dict

and part f the code that will proces all columns 

In [ ]:
# create a dict that will store outliers based on column 
outliers_by_category = {}
for numerical_column in df.select_dtypes(include='number').columns:
    if numerical_column not in ["Customer Rating", "Seller Rating", "Discount Rate"]:
        outliers_by_category[numerical_column] = {}
        for category_column in ['Category']:
            outliers_by_category[numerical_column][category_column] = identify_outliers_by_category(df, category_column, numerical_column)
                                                                           

# Displaying outliers for each category
for numerical_column, category_dict in outliers_by_category.items():
    print(f"\nOutliers in {numerical_column} by Category:")
    for category_column, outliers in category_dict.items():
        if outliers:
            for cat, out_vals in outliers.items():
                print(f"Outliers in {cat}: {out_vals.values}")
        else:
            print(f"No outliers in {category_column}")

# Replacing outliers by mean of the specific columns within each category
for numerical_column, category_dict in outliers_by_category.items():
    for category_column, outliers in category_dict.items():
        for cat, out_vals in outliers.items():
            category_mean = df.loc[df[category_column] == cat, numerical_column].mean().astype(int)
            df.loc[(df[category_column] == cat) & df[numerical_column].isin(out_vals), numerical_column] = category_mean


it is done

let's check datatypes of colums

In [ ]:
show_data_type(df)

so, let's make all numeric column in one type float64

In [ ]:
# Get a list of columns to convert to float (excluding datetime and object columns)
columns_to_convert = df.select_dtypes(include=['int']).columns
columns_to_exclude = df.select_dtypes(exclude=['datetime', 'object']).columns

# Convert integer columns to float
df[columns_to_convert] = df[columns_to_convert].astype(float)

review the change

In [ ]:
show_data_type(df)

Year isn't supposed to be float, is should be integer

In [ ]:
df['Year'] = df['Year'].astype(int)

In [ ]:
show_data_type(df)

save a new data in csv and excel files

In [ ]:
# Save data to Excel file and csv file
csv_filename = '/home/master/pycharm_projects/amazon_products/data/filtered/filtered_amazon.csv'
df.to_csv(csv_filename, index=False)

csv_filename = '/home/master/pycharm_projects/amazon_products/data/filtered/filtered_amazon.xlsx'
df.to_excel(csv_filename, index=False)